In [1]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_boston,load_iris

In [2]:
lb = load_boston()
features = lb.feature_names
x_reg ,y_reg = lb.data,lb.target
x_reg = pd.DataFrame(x_reg,columns=features)
y_reg = pd.DataFrame(y_reg,columns=['target'])

In [3]:
li = load_iris()
features2 = li.feature_names
x_cls ,y_cls = li.data,li.target
x_cls = pd.DataFrame(x_cls,columns=features2)
y_cls = pd.DataFrame(y_cls,columns=['target'])

In [4]:
x_reg.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
x_cls.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

## 线性回归

普通最小二乘的系数估计依赖于**特征的独立性**。

当特征相关且设计矩阵的列之间具有**近似线性相关性**时， 设计矩阵趋于**奇异矩阵**，最小二乘估计对观测目标的随机误差高度敏感，可能产生很大的方差。

R ^ 2定义为（1- u / v），其中u是残差平方和（（y_true-y_pred）** 2）.sum（），而v是总平方和（（y_true- y_true.mean（））** 2）.sum（）（也叫真实值的离差平方和）;

In [7]:
X_reg_train,X_reg_test,y_reg_train,y_reg_test = train_test_split(x_reg,y_reg,random_state=2022,test_size=0.2)

'''
normalize=True , 在回归之前通过减去均值并除以l2范数来对回归变量X进行归一化；
如果不在LinearRegression对象里调用归一化，则使用sklearn.preprocessing.StandardScaler
'''
reg = LinearRegression(normalize=True) 

# 通过 sample_weight 的平方根重新缩放数据样本
reg.fit(X_reg_train,y_reg_train,sample_weight=0.25)
print(f'系数：{reg.coef_} \n 截距：{reg.intercept_}')
print(f'训练集的r2:{reg.score(X_reg_train,y_reg_train)}')

y_reg_pred = reg.predict(X_reg_test)
print(f'测试集的r2:{r2_score(y_reg_test,y_reg_pred)}')

系数：[[-1.09358524e-01  4.17604608e-02  4.92275845e-02  2.75483861e+00
  -1.37425402e+01  4.71242684e+00 -1.19656681e-02 -1.43522444e+00
   2.98673854e-01 -1.23637115e-02 -8.95540107e-01  1.03495304e-02
  -5.21139480e-01]] 
 截距：[27.61502321]
训练集的r2:0.7560832029228641
测试集的r2:0.622568759700079


## 岭回归 ridge

线性回归基础上 + **L2正则项**

![20220706210934](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220706210934.png)

In [8]:
from sklearn.linear_model import Ridge

In [9]:
Ridge()


Ridge()